In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

/home/raroog/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class TrialSolution(tf.keras.models.Model):
  def __init__(self, conditions, n_i, n_h, n_o=1, activation='sigmoid', equation_type='ODE'):
    super(TrialSolution, self).__init__()
    self.n_i = n_i
    self.n_h = n_h
    self.n_o = n_o
    self.conditions = conditions
    self.hidden_layer = tf.keras.layers.Dense(units=self.n_h, activation=activation)
    self.output_layer = tf.keras.layers.Dense(units=self.n_o, activation='linear')
    
  def call(self, X):
    with tf.GradientTape() as tape:
      X = tf.convert_to_tensor(X)
      response = self.hidden_layer(X)
      response = self.output_layer(response)
      X1 = tf.concat([tf.reshape(X[:,0], shape=(X.shape[0], 1)),
                      tf.constant(1.0, dtype='float64', shape=(X.shape[0], 1))], axis=1)
#       print(X1)
      tape.watch(X1)
      response1 = self.hidden_layer(X1)
#       print(response1)
      response1 = self.output_layer(response1)
    der_resp1 = tape.gradient(response1, X1)
    print(der_resp1)
    der_resp1 = tf.reshape(der_resp1[:,1], shape=(response.shape[0],1))
#     print(der_resp1)
    x = tf.reshape(X[:,0], shape=(response.shape[0],1))
    y = tf.reshape(X[:,1], shape=(response.shape[0],1))
    one = tf.constant(1., dtype='float64', shape=(response.shape[0],1))
    two = tf.constant(2., dtype='float64', shape=(response.shape[0],1))
    pi = tf.constant(np.pi, dtype='float64', shape=(response.shape[0],1))
#     response -= response1 + der_resp1
#     print(response)
    response *= x*(one-x)*y
    response += y * two * tf.sin(pi * x)
    
#     print(response)
#     boundary_value = tf.constant(0., dtype='float64', shape=(response.shape[0],1))
    
#     for condition in self.conditions:
#       vanishing = tf.constant(1., dtype='float64', shape=(response.shape[0],1))
#       temp_bc = 0
#       if condition['type'] == 'dirichlet':
#         temp_bc = tf.reshape(condition['function'](X), shape=(response.shape[0],1))           
#         for vanisher in self.conditions:
#           if vanisher['variable'] != condition['variable'] and vanisher['value'] != condition['value']:
#             if vanisher['type'] == 'dirichlet':
#               vanishing *= (X[:, vanisher['variable']]
#                                         - tf.constant(vanisher['value'], dtype='float64', shape=(response.shape[0],1)))
#             elif vanisher['type'] == 'neumann':
#               vanishing *= (X[:, vanisher['variable']]
#                                         - tf.constant(vanisher['value'], dtype='float64', shape=(response.shape[0],1)))
#         boundary_value += temp_bc * vanishing
#         response *= (tf.constant(condition['value'], dtype='float64', shape=(response.shape[0],1))
#                      - tf.reshape(X[:, condition['variable']], shape=(response.shape[0],1)))
#       elif condition['type'] == 'neumann':
#         temp_bc = (tf.reshape(condition['function'](X), shape=(response.shape[0],1))
#                    * tf.reshape(X[:, condition['variable']], shape=(response.shape[0],1)))
#         boundary_value = temp_bc
#         response *= (tf.constant(condition['value'], dtype='float64', shape=(response.shape[0],1))
#                      - tf.reshape(X[:, condition['variable']], shape=(response.shape[0],1)))  
#     response += boundary_value
    return response

In [3]:
bcs = [{'variable':0, 'value':0, 'type':'dirichlet',
        'function':lambda X: X[:,1]**3},
        {'variable':0, 'value':0, 'type':'dirichlet',
        'function':lambda X: (tf.constant(1., dtype='float64', shape=(X.shape[0],1))+ X[:,1]**3)*tf.exp(tf.constant(-1., dtype='float64', shape=(X.shape[0],1)))},
        {'variable':1, 'value':0, 'type':'dirichlet',
        'function':lambda X: X[:,0]*tf.exp(-X[:,0])},
        {'variable':1, 'value':1, 'type':'dirichlet',
        'function':lambda X: (X[:,0]+tf.constant(1., dtype='float64', shape=(X.shape[0],1)))*tf.exp(-X[:,0])}]

In [4]:
n_samples = 100
X_p = np.linspace(0, 1, n_samples)
Y_p = np.linspace(0, 1, n_samples)
X_p, Y_p = np.meshgrid(X_p, Y_p)
X_p = X_p.flatten()
Y_p = Y_p.flatten()
samples = np.array([X_p, Y_p]).T

In [5]:
ts = TrialSolution(conditions=bcs, n_i=2, n_h=10, n_o=1)

In [6]:
def diff_loss(network, inputs):
  with tf.GradientTape() as tape2:
    with tf.GradientTape() as tape:
      inputs = tf.convert_to_tensor(inputs)
      tape.watch(inputs)
      tape2.watch(inputs)
      response = network(inputs)  
    grads = tape.gradient(response, inputs)
  laplace = tape2.gradient(grads, inputs)
#   print(grads)
#   print(laplace)
  two = tf.constant(2, dtype='float64')
  pi = tf.constant(np.pi, dtype='float64')
  loss = tf.square(laplace[:,0] + laplace[:,1]
                   - tf.sin(pi*inputs[:,0])*(two  - pi**2*inputs[:,1]**2))
  return loss

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)
train_loss = tf.keras.metrics.Mean('train')

In [8]:
@tf.function
def train_step(X):
  with tf.GradientTape() as tape:
    loss = diff_loss(ts, X)
  gradients = tape.gradient(loss, ts.trainable_variables)
  optimizer.apply_gradients(zip(gradients, ts.trainable_variables))
  train_loss(loss)

In [ ]:
ts(tf.convert_to_tensor(samples))

tf.Tensor(
[[0.10406493 0.1499706 ]
 [0.10396423 0.14985708]
 [0.10386284 0.14974278]
 ...
 [0.09146353 0.1358385 ]
 [0.09130799 0.13566561]
 [0.09115203 0.13549228]], shape=(10000, 2), dtype=float64)


<tf.Tensor: id=135, shape=(10000, 1), dtype=float64, numpy=
array([[0.00000000e+00],
       [0.00000000e+00],
       [0.00000000e+00],
       ...,
       [1.28239422e-01],
       [6.41680613e-02],
       [2.44929360e-16]])>

In [ ]:
EPOCHS = 100000
for epoch in range(EPOCHS):
  train_step(samples)
  if (epoch+1) % 1000 == 0:
    print(f'Epoch: {epoch+1} Loss: {train_loss.result().numpy()}')

Tensor("trial_solution/MatMul_2:0", shape=(10000, 2), dtype=float64)
Tensor("trial_solution/MatMul_2:0", shape=(10000, 2), dtype=float64)
Epoch: 1000 Loss: 2.565467357635498
Epoch: 2000 Loss: 1.3352611064910889
Epoch: 3000 Loss: 0.9103174805641174
Epoch: 4000 Loss: 0.6943308115005493
Epoch: 5000 Loss: 0.5637388229370117
Epoch: 6000 Loss: 0.4759729206562042
Epoch: 7000 Loss: 0.4133424758911133
Epoch: 8000 Loss: 0.36570489406585693
Epoch: 9000 Loss: 0.3283294141292572
Epoch: 10000 Loss: 0.29834502935409546
Epoch: 11000 Loss: 0.27391543984413147
Epoch: 12000 Loss: 0.25340670347213745
Epoch: 13000 Loss: 0.2360510379076004
Epoch: 14000 Loss: 0.22125670313835144
Epoch: 15000 Loss: 0.2083158940076828
Epoch: 16000 Loss: 0.1969624161720276
Epoch: 17000 Loss: 0.1870223581790924
Epoch: 18000 Loss: 0.17817144095897675
Epoch: 19000 Loss: 0.170165553689003
Epoch: 20000 Loss: 0.16303448379039764
Epoch: 21000 Loss: 0.15658172965049744
Epoch: 22000 Loss: 0.15063360333442688
Epoch: 23000 Loss: 0.1452692

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
new_shape = int(np.sqrt(samples.shape[0]))
Ze5sol = tf.reshape(ts(samples), shape=(samples.shape[0],)).numpy()
ax.plot_surface(X=samples[:,0].reshape((new_shape, new_shape)), Y=samples[:,1].reshape((new_shape, new_shape)), Z=Ze5sol.reshape((new_shape, new_shape)), label='Numerical - Training')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Ze5anal = np.sin(np.pi*samples[:,0])*samples[:,1]**2
Ze5diff = Ze5sol - Ze5anal
ax.plot_surface(X=samples[:,0].reshape((new_shape, new_shape)), Y=samples[:,1].reshape((new_shape, new_shape)), Z=Ze5diff.reshape((new_shape, new_shape)), label='Analytic')
# plt.legend()
plt.show()

In [ ]:
np.abs(Ze5diff).mean()

In [ ]:
np.max(np.abs(Ze5diff))

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
Ze5anal = np.sin(np.pi*samples[:,0])*samples[:,1]**2
Ze5diff = Ze5sol - Ze5anal
ax.plot_surface(X=samples[:,0].reshape((new_shape, new_shape)), Y=samples[:,1].reshape((new_shape, new_shape)), Z=Ze5anal.reshape((new_shape, new_shape)), label='Analytic')
# plt.legend()
plt.show()